In [14]:
import pickle
import os

import imageio
import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
tf.compat.v1.disable_eager_execution() 
tfK = tf.keras


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
model_path = "./Models Before Checkpoints/trained_cnn_2.h5"
model = tfK.models.load_model(model_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [4]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0   

In [5]:
intermediate_layer_model = tfK.models.Model(inputs=model.input,
                                            outputs=model.get_layer("flatten_4").output)

In [6]:
with open("ordered_slices_by_patient_randsubset.pkl", "rb") as f:
    patients_pkl = pickle.load(f)
label_df = pd.read_csv("labels_cleaned.csv")
label_df["ID_nopng"] = label_df["ID"].str.replace(".png", "")

In [ ]:
label_df.describe()

In [ ]:
ID_list = label_df["ID_nopng"].tolist()
print("ID_ce6315db3" in ID_list)

In [7]:
min = 9999
lens = []
for key, item in patients_pkl.items():
    if len(item) < min:
        min = len(item)
    lens.append(len(item))

In [15]:
# remove bad slices
patients_pkl_clean = dict()
for key, item in patients_pkl.items():
    tmp = []
    for slice_id in item:
        if os.path.isfile("./Windowed-PNGs-FINAL-comb/" + slice_id + ".png"):
            tmp.append(slice_id)
    patients_pkl_clean[key] = tmp

In [16]:
n_slices = 24

patients_long_enough = dict()
for key, item in patients_pkl_clean.items():
    if len(item) >= n_slices:
        mid_slice = len(item)//2
        truncated_slice_IDs = item.copy()[mid_slice - n_slices//2:mid_slice + n_slices//2]
        patients_long_enough[key] = truncated_slice_IDs

In [17]:
n_patients = len(patients_long_enough)
n_features = 8192

len(patients_long_enough)

2418

In [35]:
patients_long_enough.keys()

dict_keys(['ID_14402092', 'ID_32e5f08b', 'ID_aa1d288b', 'ID_4151e8a0', 'ID_77759530', 'ID_ebcf8504', 'ID_42b5c14d', 'ID_95874385', 'ID_9857543b', 'ID_8c18d1e4', 'ID_0f79ec8f', 'ID_72852ec4', 'ID_623106d8', 'ID_267a0f84', 'ID_7ce98286', 'ID_14fd5765', 'ID_7bff0a1e', 'ID_ee4d33d9', 'ID_f87fb90b', 'ID_0a569ef2', 'ID_e377f42d', 'ID_17c9dd54', 'ID_6d751398', 'ID_08039d3e', 'ID_8d2a9e3a', 'ID_de8ad011', 'ID_b14f8f3b', 'ID_1d92df66', 'ID_2e9eb3f8', 'ID_02584b75', 'ID_d0e60d77', 'ID_27a4107a', 'ID_aa4cf1ec', 'ID_2e717ad5', 'ID_b583d81a', 'ID_9e87cd71', 'ID_7f853ca3', 'ID_a055cdb7', 'ID_d6aae33e', 'ID_fc8e9dff', 'ID_f977a3db', 'ID_d158fac5', 'ID_ec589610', 'ID_39974c92', 'ID_5ad01c9e', 'ID_9d857216', 'ID_0873dce9', 'ID_07018e5b', 'ID_4b1dfa3d', 'ID_19894739', 'ID_e5dc2ecf', 'ID_3aba4932', 'ID_87f66e36', 'ID_54af5c13', 'ID_8bd9b50b', 'ID_81c9a455', 'ID_ba517fda', 'ID_80ef001a', 'ID_d190fdf0', 'ID_ec5a9a79', 'ID_ed2178bf', 'ID_9d0e4e63', 'ID_4474c06a', 'ID_4977663f', 'ID_02bddf7a', 'ID_f279a2a7',

In [11]:
patient_slices = patients_long_enough['ID_32e5f08b']
print(patient_slices)

label_df[label_df["ID_nopng"].isin(patient_slices)]

['ID_651cf7f78', 'ID_fa042a1cb', 'ID_e766a373e', 'ID_04b0ae52a', 'ID_d5e59f134', 'ID_aec53523b', 'ID_f2ac96cb4', 'ID_6c8f8dc64', 'ID_15c203093', 'ID_62a79a972', 'ID_8e1ecb4c3', 'ID_c971c9888', 'ID_4bd7cc624', 'ID_1b00cdf51', 'ID_7ca7b0395', 'ID_bbafbfbb4', 'ID_868265929', 'ID_9f3f89002', 'ID_b0e7477cf', 'ID_eba4cda29', 'ID_e1b9638cc', 'ID_ba206670f', 'ID_20358afab', 'ID_9009db145']


,Unnamed: 0,ID,any,ID_nopng
12333,12333,ID_04b0ae52a.png,0,ID_04b0ae52a
57507,57507,ID_15c203093.png,0,ID_15c203093
71287,71287,ID_1b00cdf51.png,1,ID_1b00cdf51
84886,84886,ID_20358afab.png,0,ID_20358afab
199772,199772,ID_4bd7cc624.png,1,ID_4bd7cc624
260431,260431,ID_62a79a972.png,1,ID_62a79a972
266901,266901,ID_651cf7f78.png,0,ID_651cf7f78
286529,286529,ID_6c8f8dc64.png,0,ID_6c8f8dc64
328864,328864,ID_7ca7b0395.png,1,ID_7ca7b0395
354912,354912,ID_868265929.png,0,ID_868265929


In [18]:
# data_array = np.zeros([n_patients, n_slices, n_features])
# label_array = np.zeros([n_patients, n_slices, 1])
data_list = []
label_list = []

for i, (patient_ID, slice_IDs) in enumerate(tqdm.tqdm(patients_long_enough.items())):
#     if i%10 == 0:
#         print(i)
    data_patient_list = []
    label_patient_list = []
    for j, slice_ID in enumerate(slice_IDs):
        # load PNG
        png_array = np.expand_dims(imageio.imread("./Windowed-PNGs-FINAL-comb/" + slice_ID + ".png"), 0)
#         print(png_array.shape)
        layer_features = intermediate_layer_model.predict(png_array).flatten()
#         print(layer_features.shape)
#         print(layer_features.dtype)
        
#         data_array[i][j] = layer_features
        data_patient_list.append(layer_features)
        
#         print(slice_ID)
#         print(label_df[label_df["ID_nopng"]==slice_ID]["any"].iloc[0])

#         label_array[i][j] = label_df[label_df["ID_nopng"]==slice_ID]["any"].iloc[0]
        label_patient_list.append(label_df[label_df["ID_nopng"]==slice_ID]["any"].iloc[0])
    
        
#         break
#     break

    data_list.append(data_patient_list)
    label_list.append(label_patient_list)
    
data_array = np.array(data_list)
label_array = np.array(label_list)


  3%|██▏                                                                             | 65/2418 [01:25<51:14,  1.31s/it]


  5%|████▎                                                                          | 131/2418 [02:51<50:29,  1.32s/it]


  8%|██████▍                                                                        | 197/2418 [04:20<50:18,  1.36s/it]


 11%|████████▌                                                                      | 263/2418 [05:52<50:49,  1.42s/it]


 14%|██████████▋                                                                    | 329/2418 [07:27<51:14,  1.47s/it]


...

 90%|████████████████████████████████████████████████████████████████████▍       | 2177/2418 [1:08:51<09:44,  2.43s/it]


 93%|██████████████████████████████████████████████████████████████████████▍     | 2243/2418 [1:11:33<07:15,  2.49s/it]


 95%|████████████████████████████████████████████████████████████████████████▌   | 2309/2418 [1:14:17<04:35,  2.53s/it]


 98%|██████████████████████████████████████████████████████████████████████████▋ | 2375/2418 [1:17:04<01:46,  2.48s/it]


100%|████████████████████████████████████████████████████████████████████████████| 2418/2418 [1:18:53<00:00,  1.96s/it]


In [20]:
print(np.sum(label_array))
print(np.average(label_array))

11231
0.1935311552247036


In [21]:
np.save("rcnn-data-array", data_array)
np.save("rcnn-label-array", label_array)